In [ ]:
import sys

import rasterio as rio
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm

from pathlib import Path

import numpy as np

### Config

In [ ]:
dataset_path = Path('../../change_detection_dataset/SN7_buildings/train')

train_df_path = '../../change_detection_dataset/dataset_1/train.csv'
valid_df_path = '../../change_detection_dataset/dataset_1/valid.csv'
test_df_path = '../../change_detection_dataset/dataset_1/test.csv'

In [ ]:
def get_histogram(df_path):
    df = pd.read_csv(df_path, index_col=0)
    df = df[['AOI_name', 'filename']].drop_duplicates()
    
    no_of_pixels = 0
    rgb_freq = np.zeros((3, 255), dtype=np.float32)
    
    for _, row in tqdm(df.iterrows()):
        AOI_name, file_name = row
        image_path = dataset_path / AOI_name / 'images_masked' / '{}.tif'.format(file_name)
        
        r = rio.open(image_path).read()
    
        image = r.transpose((1, 2, 0))[:, :, :-1]
        freq = image[(image != [0, 0, 0]).any(axis=-1)]
        
        no_of_pixels += freq.shape[0]
        
        rgb_freq[0] += np.histogram(freq[:, 0], bins=255, range=(0, 255))[0]
        rgb_freq[1] += np.histogram(freq[:, 1], bins=255, range=(0, 255))[0]
        rgb_freq[2] += np.histogram(freq[:, 2], bins=255, range=(0, 255))[0]
    
    rgb_freq = rgb_freq / no_of_pixels

    return rgb_freq

In [ ]:
train_hist = get_histogram(train_df_path)
valid_hist = get_histogram(valid_df_path)
test_hist = get_histogram(test_df_path)

In [ ]:
fig, axs = plt.subplots()

plt.plot(train_hist[0], color='r')
plt.plot(train_hist[1], color='g')
plt.plot(train_hist[2], color='b')

plt.plot(valid_hist[0], color='r', linestyle='--')
plt.plot(valid_hist[1], color='g', linestyle='--')
plt.plot(valid_hist[2], color='b', linestyle='--')

plt.plot(test_hist[0], color='r', linestyle=':')
plt.plot(test_hist[1], color='g', linestyle=':')
plt.plot(test_hist[2], color='b', linestyle=':')

### Train

In [ ]:
fig, axs = plt.subplots()

plt.plot(train_hist[0], color='r')
plt.plot(train_hist[1], color='g')
plt.plot(train_hist[2], color='b')

### Valid

In [ ]:
fig, axs = plt.subplots()

plt.plot(valid_hist[0], color='r')
plt.plot(valid_hist[1], color='g')
plt.plot(valid_hist[2], color='b')

### Test

In [ ]:
fig, axs = plt.subplots()

plt.plot(test_hist[0], color='r')
plt.plot(test_hist[1], color='g')
plt.plot(test_hist[2], color='b')